In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb

/home/jiashen/env3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
train = pd.read_csv('train_clean.csv')
test = pd.read_csv('test_clean.csv')

In [4]:
fullsq_fill = np.mean(train[train['full_sq'].notnull()]['full_sq'])
train['full_sq'] = train['full_sq'].fillna(fullsq_fill)
fullsq_fill = np.mean(test[test['full_sq'].notnull()]['full_sq'])
test['full_sq'] = test['full_sq'].fillna(fullsq_fill)

In [5]:
Price_sqm = [x/y for x,y in zip(list(train['price_doc']*0.95+10),list(train['full_sq']))]

In [6]:
train['Price_sqm'] = Price_sqm

In [7]:
train['life_sq_ratio'] = train['life_sq']/train['full_sq']
train['kitch_sq_ratio'] = train['kitch_sq']/train['full_sq']
test['life_sq_ratio'] = test['life_sq']/test['full_sq']
test['kitch_sq_ratio'] = test['kitch_sq']/test['full_sq']

In [8]:
col = list(test.columns)[2:]
col = [each for each in col if each not in ['full_sq','life_sq','kitch_sq']]
label = train['Price_sqm']

In [9]:
dtrain = xgb.DMatrix(train[col],label)

In [10]:
params = {
   'eta': 0.05, ## Try 0.01,3,5
   'max_depth': 5,## Try 4,5,6
   'subsample': 0.7,
   'colsample_bytree': 0.7,
   'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}
xgb_cvalid = xgb.cv(params, dtrain, num_boost_round=1000, early_stopping_rounds=20,
     verbose_eval=50, show_stdv=True,seed=42)
print(len(xgb_cvalid))

[0]	train-rmse:133514+614.879	test-rmse:133512+1250.17
[50]	train-rmse:34906.2+508.618	test-rmse:37402.7+1748.64
[100]	train-rmse:30887.3+338.079	test-rmse:35262.3+1798.06
[150]	train-rmse:29474.8+330.452	test-rmse:35047.9+1865.59
[200]	train-rmse:28375+398.788	test-rmse:34936.3+1866.63
[250]	train-rmse:27454.8+379.765	test-rmse:34875.4+1886.71
275


In [11]:
dtest = xgb.DMatrix(test[col])

In [12]:
model = xgb.train(params,dtrain,num_boost_round=len(xgb_cvalid))

In [13]:
pred = model.predict(dtest)*test['full_sq']

In [15]:
sub = pd.DataFrame({'id':test['id'],'price_doc':pred})

In [16]:
sub.to_csv('Price_SQRM_as_Label_With_Basic_Features_0.95label.csv',index=False)